
# TF-recomm


In [5]:
import time
from collections import deque
import socket
import sys
import numpy as np
import tensorflow as tf
from six import next
from tensorflow.core.framework import summary_pb2
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file

import dataio
import ops
# tensorboard --logdir=.\_zfp\data\my_graph
# tensorboard => http://localhost:6006 
# jupyter => http://localhost:8889

In [2]:
np.random.seed(13575)

BATCH_SIZE = 1000
USER_NUM = 6040
ITEM_NUM = 3952
DIM = 15
EPOCH_MAX = 100
DEVICE = "/cpu:0"

In [4]:
def clip(x):
    return np.clip(x, 1.0, 5.0)


def make_scalar_summary(name, val):
    return summary_pb2.Summary(value=[summary_pb2.Summary.Value(tag=name, simple_value=val)])

def get_data():
    df = dataio.read_process("./tmp/movielens/ml-1m/ratings.dat", sep="::")
    rows = len(df)
    df = df.iloc[np.random.permutation(rows)].reset_index(drop=True)
    split_index = int(rows * 0.9)
    df_train = df[0:split_index]
    df_test = df[split_index:].reset_index(drop=True)
    return df_train, df_test, rows

def get_movies():
    df = dataio.read_movies("./tmp/movielens/ml-1m/movies.dat", sep="::")
    rows = len(df)
    return df, rows

# Data

In [6]:
df_train, df_test, length = get_data()
df_movies,rows = get_movies()

Movies file length:
3883
Toy Story (1995)
1
Animation|Children's|Comedy


In [7]:
df_train.iloc[0:10]
df_train.describe()

,user,item,rate,st
count,900188.000000,900188.000000,900188.000000,9.001880e+05
mean,3022.600601,1864.635758,3.581352,9.722461e+08
std,1728.384446,1096.003424,1.117216,1.215377e+07
min,0.000000,0.000000,1.000000,9.567039e+08
25%,1504.000000,1029.000000,3.000000,9.653027e+08
50%,3068.000000,1834.000000,4.000000,9.730215e+08
75%,4475.000000,2769.000000,4.000000,9.752212e+08
max,6039.000000,3951.000000,5.000000,1.046455e+09


In [12]:
print ("Movies file length: {}" .format(len(df_movies)))

Movies file length: 3883


In [13]:
df_movies.iloc[0:10]

,movie,title,tags
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [10]:
df_movies.iloc[0].title

'Toy Story (1995)'

In [15]:
# type(df_movies)

# Network + train + test

In [16]:

samples_per_batch = len(df_train) // BATCH_SIZE

iter_train = dataio.ShuffleIterator([df_train["user"],
                                     df_train["item"],
                                    df_train["rate"]],
                                    batch_size=BATCH_SIZE)

iter_test = dataio.OneEpochIterator([df_test["user"],
                                     df_test["item"],
                                    df_test["rate"]],
                                    batch_size=-1)

user_batch = tf.placeholder(tf.int32, shape=[None], name="id_user")
item_batch = tf.placeholder(tf.int32, shape=[None], name="id_item")
rate_batch = tf.placeholder(tf.float32, shape=[None])

infer, regularizer = ops.inference_svd(user_batch, item_batch, user_num=USER_NUM, item_num=ITEM_NUM, dim=DIM, device=DEVICE)
global_step = tf.contrib.framework.get_or_create_global_step()
_, train_op = ops.optimization(infer, regularizer, rate_batch, learning_rate=0.001, reg=0.05, device=DEVICE)
#zeros= tf.Variable(tf.zeros([1]),name="zeros")


In [17]:

def svd(train, test,length,moviefile, trainFl=False):
    init_op = tf.global_variables_initializer()
    saver=tf.train.Saver()
    with tf.Session() as sess:
        sess.run(init_op)
        if trainFl == True: 
            summary_writer = tf.summary.FileWriter(logdir="./tmp/svd/log", graph=sess.graph)
            print("{} {} {} {}".format("epoch", "train_error", "val_error", "elapsed_time"))
            errors = deque(maxlen=samples_per_batch)
            start = time.time()
            for i in range(EPOCH_MAX * samples_per_batch):
                users, items, rates = next(iter_train)
                _, pred_batch = sess.run([train_op, infer], feed_dict={user_batch: users,
                                                                    item_batch: items,
                                                                                                                        rate_batch: rates})
                pred_batch = clip(pred_batch)
                errors.append(np.power(pred_batch - rates, 2))
                if i % samples_per_batch == 0:
                    train_err = np.sqrt(np.mean(errors))
                    test_err2 = np.array([])
                    for users, items, rates in iter_test:
                        pred_batch = sess.run(infer, feed_dict={user_batch: users,
                                                                item_batch: items})
                        pred_batch = clip(pred_batch)
                        test_err2 = np.append(test_err2, np.power(pred_batch - rates, 2))
                    end = time.time()
                    test_err = np.sqrt(np.mean(test_err2))
                    print("{:3d} {:f} {:f} {:f}(s)".format(i // samples_per_batch, train_err, test_err,
                                                        end - start))
                    train_err_summary = make_scalar_summary("training_error", train_err)
                    test_err_summary = make_scalar_summary("test_error", test_err)
                    summary_writer.add_summary(train_err_summary, i)
                    summary_writer.add_summary(test_err_summary, i)
                    start = end

            #meta_graph_def = tf.train.export_meta_graph(filename='/tmp/tfrecomm.meta')
            save_path=saver.save(sess,"./tmp/")
        else: 
            print("model restored")
            saver.restore(sess, "./tmp/")

In [18]:
svd(df_train, df_test, length,df_movies, trainFl=False) 
print("Done!")

model restored
INFO:tensorflow:Restoring parameters from ./tmp/
Done!


# EXECUTION

In [32]:
def printMM(topmovies):
    print (topmovies)
    print("TOP Movies")
    for i,r in topmovies:
        print("{0:5} - {1:1.2f} - {2}" .format(i,  r, df_movies.iloc[i].title  ))

In [44]:
def test(train, test,length,moviefile, data, trainFl=False):
    init_op = tf.global_variables_initializer()
    #saver=tf.train.Saver()
    with tf.Session() as sess:
        sess.run(init_op)
        #saver.restore(sess, "./tmp/")

        movies=list(range(len(moviefile)))
        #print (movies)
        users=[1]
        pred_batch = sess.run(infer, feed_dict={user_batch: users,item_batch: movies})
        moviesrecomm=list(zip(movies,pred_batch))
        smovies=sorted (moviesrecomm,key=lambda x:x[1],reverse=True)

        print ("\nTop Movies ------------------------------------------------------------")
        topmovies= smovies[0:10]
        #print (topmovies)
        printMM(topmovies)
        
        #-----------------------------------------------------------------------------
        
        print ("\n User - data {} ------------------------------------------------------------\n" .format(data))
        # give number between 1 - 5000
        del users[:]
        users.append(int(data))
        pred_batch = sess.run(infer, feed_dict={user_batch: users,item_batch: movies})
        moviesrecomm=list(zip(movies,pred_batch))
        smovies=sorted (moviesrecomm,key=lambda x:x[1],reverse=True)
        topmovies= smovies[0:10]
        printMM(topmovies)
        for item in topmovies:
            itopmovie=item[0]
            recommendedmovie=moviefile["title"][itopmovie]
            recommendedtags=moviefile["tags"][itopmovie]
    return

In [45]:
test(df_train, df_test, length,df_movies, data=2, trainFl=False) 


Top Movies ------------------------------------------------------------
[(3047, 0.68861681), (1702, 0.68797344), (2616, 0.68785566), (2799, 0.6878379), (2998, 0.68755776), (1495, 0.68749475), (2989, 0.68738717), (1636, 0.68736839), (250, 0.687168), (1370, 0.68716365)]
TOP Movies
 3047 - 0.69 - Miss Julie (1999)
 1702 - 0.69 - Shooting Fish (1997)
 2616 - 0.69 - Red Dwarf, The (Le Nain rouge) (1998)
 2799 - 0.69 - Fright Night Part II (1989)
 2998 - 0.69 - Women on the Verge of a Nervous Breakdown (1988)
 1495 - 0.69 - Sprung (1997)
 2989 - 0.69 - Ape, The (1940)
 1636 - 0.69 - Truman Show, The (1998)
  250 - 0.69 - Interview with the Vampire (1994)
 1370 - 0.69 - Mars Attacks! (1996)

 User - data 2 ------------------------------------------------------------

[(871, 0.70792109), (1849, 0.70786357), (1670, 0.70767885), (2933, 0.70740688), (1714, 0.70718849), (505, 0.70697927), (91, 0.706738), (3585, 0.70666087), (1710, 0.70661807), (2515, 0.70649517)]
TOP Movies
  871 - 0.71 - Trigger

# TO DO: 
* Read Users 
* Compare results for similar users 
* Show statistics


## Users Information
- Gender is denoted by a "M" for male and "F" for female
- Age is chosen from the following ranges:

	*  1:  "Under 18"
	* 18:  "18-24"
	* 25:  "25-34"
	* 35:  "35-44"
	* 45:  "45-49"
	* 50:  "50-55"
	* 56:  "56+"

- Occupation is chosen from the following choices:
	*  0:  "other" or not specified
	*  1:  "academic/educator"
	*  2:  "artist"
	*  3:  "clerical/admin"
	*  4:  "college/grad student"
	*  5:  "customer service"
	*  6:  "doctor/health care"
	*  7:  "executive/managerial"
	*  8:  "farmer"
	*  9:  "homemaker"
	* 10:  "K-12 student"
	* 11:  "lawyer"
	* 12:  "programmer"
	* 13:  "retired"
	* 14:  "sales/marketing"
	* 15:  "scientist"
	* 16:  "self-employed"
	* 17:  "technician/engineer"
	* 18:  "tradesman/craftsman"
	* 19:  "unemployed"
	* 20:  "writer"

In [1]:
import pandas as pd

In [9]:
def get_users():
    col_names = ["userID", "gender", "age", "occupation", "zipcode"]
    df = pd.read_csv("./tmp/movielens/ml-1m/users.dat", sep="::", header=None, names=col_names, engine='python')
    rows = len(df)
    return df, rows


In [10]:
df_users,rows = get_users()


In [11]:
print(len(df_users))
df_users.describe()

6040


,userID,age,occupation
count,6040.000000,6040.000000,6040.000000
mean,3020.500000,30.639238,8.146854
std,1743.742145,12.895962,6.329511
min,1.000000,1.000000,0.000000
25%,1510.750000,25.000000,3.000000
50%,3020.500000,25.000000,7.000000
75%,4530.250000,35.000000,14.000000
max,6040.000000,56.000000,20.000000


In [12]:
df_users.iloc[0:10]

,userID,gender,age,occupation,zipcode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


## FIND similar users 


In [26]:
def get_users_where(age="*", gender="*", occupation="*", zipcode="*"): 
    us = df_users
    if age != "*":
        us = us[us["age"]==age]
    if gender != "*":
        us = us[us["gender"]==gender]
    if occupation != "*":
        us = us[us["occupation"]==occupation]
    if zipcode != "*":
        us = us[us["zipcode"]==zipcode]    
    return us # return the data 
#     return len(us) return the length of each selection

# print(get_users_where(age=1) )
# print(get_users_where(age=1, gender ="M") )
print(get_users_where(age=1, gender ="M", occupation = 1 ) )
# print(get_users_where(age=1, gender ="M", occupation = 1, zipcode = 48067 ) )


      userID gender  age  occupation zipcode
1814    1815      M    1           1   30707
2059    2060      M    1           1   48304
2852    2853      M    1           1  444555


# Compare results - Show statistics